# Lab 3: Clonal deconvolution

Implementation in Python of a combinatorial algorithm for clonal deconvolution.

M. Tellaetxe-Abete, B. Calvo and C. Lawrie, *"An Iterated Local Search Algorithm for the Clonal Deconvolution Problem," 2020 IEEE Congress on Evolutionary Computation (CEC)*, 2020, pp. 1-8, doi: 10.1109/CEC48606.2020.9185515.

## About

**Course**  
Bioinformatics and Statistical Genetics (BSG-MIRI)  
FIB - Universitat Politècnica de Catalunya. BarcelonaTech  
Autumn 2021  

**Team**  
* Antoni Casas
&lt;antoni.casas.munoz@estudiantat.upc.edu&gt;
* Marcel Cases
&lt;marcel.cases@estudiantat.upc.edu&gt;

## The algorithm

In [1]:
import numpy as np
from random import randrange

# F = np.matrix("0.4 0.4 0.4 0.0 0.0 0.0; 0.3 0.3 0.0 0.3 0.0 0.0; 0.4 0.0 0.0 0.0 0.3 0.2")
F = np.matrix("0.4 0.0 0.0 0.0 0.4 0.4; 0.3 0.0 0.4 0.3 0.0 0.3; 0.0 0.3 0.4 0.4 0.3 0.2")
n, m = np.shape(F)
k = m

def Uprime(U):
    for i in range(0,n):
        U[i][U[i]<0] = 0
        U[i]=U[i]/np.sum(U[i])
    return U

#Perform graft to node idx of node i
def graft(idx,i,Bcopy):
    children=Bcopy[:,i]==1
    oldNode=Bcopy[i].copy()
    mutations=Bcopy[idx].copy()
    mutations[i]=1
    Bcopy[i]=mutations.copy()
    valuesToChange=[]
    for idx,val in enumerate(mutations):
        if val!=oldNode[idx]:
            valuesToChange.append((idx,val))
    childTuples=[]
    for idx,elem in enumerate(children):
        if elem:
            childTuples.append(idx)
    for elem in childTuples:
        oldMut=Bcopy[elem]
        for val in valuesToChange:
            oldMut[val[0]]=val[1]

#Perturbation
def changeRandomRoot(B):
    num_rows, num_cols = B.shape
    newRoot=randrange(num_rows)
    B[newRoot]=np.zeros(num_cols)
    for row in range(num_rows):
        B[row][newRoot]=1
        
def evaluate(F,B):
    Binv = np.linalg.inv(B) # always invertible
    U=F*Binv
    Up=Uprime(U)
    absSum=np.sum(np.absolute(F-Up*B)) # obj. function
    num_rows, num_cols = F.shape
    return absSum/(num_rows*num_cols) # obj. function

In [2]:
B=np.zeros((k,m))

#Starting B, simplest possible tree
for i in range(0,k):
    B[i][i]=1
    for j in range(0,i+1):
        B[i][j]=1
    
bestLoss=np.inf
bestLossAbs=np.inf
bestB=None
consecutiveFail=0
print("\n=== Local Search start ===")
while bestLoss>0.0001 and consecutiveFail<1000:
    for i in range(0,k):
        possibleGrafts=B[:,i]==0
        improvement=False
        for idx,elem in enumerate(possibleGrafts):
            if elem:
                Bcopy=B.copy()
                graft(idx,i,Bcopy)
                loss=evaluate(F,Bcopy)
                if loss<bestLoss:
                    if loss<bestLossAbs:
                        bestLossAbs=loss
                        print("Best loss\t", bestLossAbs)
                        bestB=Bcopy.copy()
                        improvement=True
                        consecutiveFail=0
                    bestB=Bcopy
                    B=Bcopy
                    bestLoss=loss
    if not improvement:
        bestLoss=np.inf
        changeRandomRoot(B)
        consecutiveFail+=1

print("\nBest loss", bestLossAbs)
print("\nBest clone genotype matrix B\n", bestB)


=== Local Search start ===
Best loss	 0.4583333333333333
Best loss	 0.3277777777777777
Best loss	 0.2080808080808081
Best loss	 0.1871794871794872
Best loss	 0.16666666666666666
Best loss	 0.15940170940170942
Best loss	 0.15000000000000002
Best loss	 0.1462962962962963
Best loss	 0.1444444444444445
Best loss	 0.13484848484848486

Best loss 0.13484848484848486

Best clone genotype matrix B
 [[1. 0. 1. 1. 0. 0.]
 [0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1.]]


In [3]:
Binv = np.linalg.inv(bestB) 
U=F*Binv
Up=Uprime(U)
absSum=np.sum(np.absolute(F-Up*B))

print("\nClone frequency matrix U\n", Up)


Clone frequency matrix U
 [[0.5        0.         0.         0.         0.5        0.        ]
 [0.42857143 0.         0.14285714 0.         0.         0.42857143]
 [0.         0.5        0.         0.         0.5        0.        ]]


In [4]:
def writeTree(toFindRoot,B,currentLevel,currentNode,root):
    isroot=False
    if currentNode==root:
        isroot=True
        ETEstring=""
    else:
        ETEstring=str(currentNode)
    first=True
    children=0
    ETEstringAccum=""
    for elem in toFindRoot:
        if elem[1]==currentLevel and B[elem[0]][currentNode]==1:
            children+=1
            if first and children>1:
                if not isroot:
                    ETEstring+=",("
                first=False
                ETEstringAccum=ETEstringAccum.replace(",","",1)
            ETEstringAccum+=","+writeTree(toFindRoot,B,currentLevel+1,elem[0],root)
    ETEstring+=ETEstringAccum
    if children>1:
        if not isroot:
            ETEstring+=")"
    return ETEstring

In [5]:
from ete3 import Tree

t = Tree()
toFindRoot=[]
num_rows, num_cols = bestB.shape

for row in range(0,num_rows):
    toFindRoot.append((row,sum(bestB[row,:])))
toFindRoot.sort(key=lambda x:x[1])
root=toFindRoot[0][0]
del toFindRoot[0]
print("\nBest clone genotype matrix B\n", bestB)
tree="("+writeTree(toFindRoot,bestB,2,root,root)+")"+str(root)+";"
tree=tree.replace("(,","(")

print("\nTree (Newick)\n", tree)
t = Tree(tree)
print("\nTree", t)


Best clone genotype matrix B
 [[1. 0. 1. 1. 0. 0.]
 [0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1.]]

Tree (Newick)
 (3,(0,1,5,4))2;

Tree 
   /-3
  |
--|   /-0
  |  |
  |  |--1
   \-|
     |--5
     |
      \-4
